In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.functions import to_timestamp,to_date,date_format
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import unix_timestamp

spark = SparkSession.builder.getOrCreate()

In [ ]:
df = spark.read.csv(f"/content/drive/MyDrive/btc.csv", inferSchema = True, header = True)

In [ ]:
df.printSchema()

root
 |-- time: timestamp (nullable = true)
 |--  id: integer (nullable = true)
 |--  amount: double (nullable = true)
 |--  open: double (nullable = true)
 |--  high: double (nullable = true)
 |--  low: double (nullable = true)
 |--  close: double (nullable = true)
 |--  count: double (nullable = true)
 |--  vol: double (nullable = true)



In [ ]:
from pyspark.sql.functions import to_timestamp

# assuming you have a PySpark DataFrame called 'df'
df = df.withColumn('time', to_timestamp('time', 'yyyy-MM-dd HH:mm:ss'))

In [ ]:
df.printSchema()

root
 |-- time: timestamp (nullable = true)
 |--  id: integer (nullable = true)
 |--  amount: double (nullable = true)
 |--  open: double (nullable = true)
 |--  high: double (nullable = true)
 |--  low: double (nullable = true)
 |--  close: double (nullable = true)
 |--  count: double (nullable = true)
 |--  vol: double (nullable = true)



In [ ]:
df.show()

+-------------------+----------+------------------+-------+-------+-------+-------+------+------------------+
|               time|        id|            amount|   open|   high|    low|  close| count|               vol|
+-------------------+----------+------------------+-------+-------+-------+-------+------+------------------+
|2020-01-01 00:00:00|1577808000|102.74380394536281| 7194.7| 7200.0|7186.19| 7200.0| 497.0| 738706.3461761171|
|2020-01-01 00:01:00|1577808060| 70.71166088549622| 7200.0|7219.58|7199.99| 7218.2| 448.0| 509693.2114912161|
|2020-01-01 00:02:00|1577808120|17.445959611852214|7218.21|7218.44|7213.09|7216.22| 248.0|   125889.77536392|
|2020-01-01 00:03:00|1577808180| 13.03460623212044|7216.21|7221.91|7215.48|7218.37| 300.0| 94091.10083095604|
|2020-01-01 00:04:00|1577808240|14.004799868097122|7218.37|7220.13|7210.01|7213.05| 279.0|   101042.53180219|
|2020-01-01 00:05:00|1577808300| 12.57289432271876|7213.48|7221.76|7212.62|7218.38| 189.0| 90727.99216263865|
|2020-01-0

In [ ]:
from pyspark.sql.functions import isnull, when, col, count, lit, regexp_extract, mean, avg, split
df.select([count(when(isnull(c),c)).alias(c) for c in df.columns]).show()

+----+---+-------+-----+-----+----+------+------+----+
|time| id| amount| open| high| low| close| count| vol|
+----+---+-------+-----+-----+----+------+------+----+
|   0|  0|      0|    0|    0|   0|     0|     0|   0|
+----+---+-------+-----+-----+----+------+------+----+



In [ ]:
df.printSchema()

root
 |-- time: timestamp (nullable = true)
 |--  id: integer (nullable = true)
 |--  amount: double (nullable = true)
 |--  open: double (nullable = true)
 |--  high: double (nullable = true)
 |--  low: double (nullable = true)
 |--  close: double (nullable = true)
 |--  count: double (nullable = true)
 |--  vol: double (nullable = true)



In [ ]:
df.show()

+-------------------+----------+------------------+-------+-------+-------+-------+------+------------------+
|               time|        id|            amount|   open|   high|    low|  close| count|               vol|
+-------------------+----------+------------------+-------+-------+-------+-------+------+------------------+
|2020-01-01 00:00:00|1577808000|102.74380394536281| 7194.7| 7200.0|7186.19| 7200.0| 497.0| 738706.3461761171|
|2020-01-01 00:01:00|1577808060| 70.71166088549622| 7200.0|7219.58|7199.99| 7218.2| 448.0| 509693.2114912161|
|2020-01-01 00:02:00|1577808120|17.445959611852214|7218.21|7218.44|7213.09|7216.22| 248.0|   125889.77536392|
|2020-01-01 00:03:00|1577808180| 13.03460623212044|7216.21|7221.91|7215.48|7218.37| 300.0| 94091.10083095604|
|2020-01-01 00:04:00|1577808240|14.004799868097122|7218.37|7220.13|7210.01|7213.05| 279.0|   101042.53180219|
|2020-01-01 00:05:00|1577808300| 12.57289432271876|7213.48|7221.76|7212.62|7218.38| 189.0| 90727.99216263865|
|2020-01-0

In [ ]:
#df = df.drop("time")

In [ ]:
df.show()

+-------------------+----------+------------------+-------+-------+-------+-------+------+------------------+
|               time|        id|            amount|   open|   high|    low|  close| count|               vol|
+-------------------+----------+------------------+-------+-------+-------+-------+------+------------------+
|2020-01-01 00:00:00|1577808000|102.74380394536281| 7194.7| 7200.0|7186.19| 7200.0| 497.0| 738706.3461761171|
|2020-01-01 00:01:00|1577808060| 70.71166088549622| 7200.0|7219.58|7199.99| 7218.2| 448.0| 509693.2114912161|
|2020-01-01 00:02:00|1577808120|17.445959611852214|7218.21|7218.44|7213.09|7216.22| 248.0|   125889.77536392|
|2020-01-01 00:03:00|1577808180| 13.03460623212044|7216.21|7221.91|7215.48|7218.37| 300.0| 94091.10083095604|
|2020-01-01 00:04:00|1577808240|14.004799868097122|7218.37|7220.13|7210.01|7213.05| 279.0|   101042.53180219|
|2020-01-01 00:05:00|1577808300| 12.57289432271876|7213.48|7221.76|7212.62|7218.38| 189.0| 90727.99216263865|
|2020-01-0

In [ ]:
df = df.drop(" id")

In [ ]:
df.show()

+-------------------+------------------+-------+-------+-------+-------+------+------------------+
|               time|            amount|   open|   high|    low|  close| count|               vol|
+-------------------+------------------+-------+-------+-------+-------+------+------------------+
|2020-01-01 00:00:00|102.74380394536281| 7194.7| 7200.0|7186.19| 7200.0| 497.0| 738706.3461761171|
|2020-01-01 00:01:00| 70.71166088549622| 7200.0|7219.58|7199.99| 7218.2| 448.0| 509693.2114912161|
|2020-01-01 00:02:00|17.445959611852214|7218.21|7218.44|7213.09|7216.22| 248.0|   125889.77536392|
|2020-01-01 00:03:00| 13.03460623212044|7216.21|7221.91|7215.48|7218.37| 300.0| 94091.10083095604|
|2020-01-01 00:04:00|14.004799868097122|7218.37|7220.13|7210.01|7213.05| 279.0|   101042.53180219|
|2020-01-01 00:05:00| 12.57289432271876|7213.48|7221.76|7212.62|7218.38| 189.0| 90727.99216263865|
|2020-01-01 00:06:00|         26.494623|7218.69|7218.69|7213.55|7216.32| 120.0|    191226.4272875|
|2020-01-0

In [ ]:
df.printSchema()

root
 |-- time: timestamp (nullable = true)
 |--  amount: double (nullable = true)
 |--  open: double (nullable = true)
 |--  high: double (nullable = true)
 |--  low: double (nullable = true)
 |--  close: double (nullable = true)
 |--  count: double (nullable = true)
 |--  vol: double (nullable = true)



In [ ]:
from pyspark.sql.functions import year, month, dayofmonth

# Assuming our dataframe is named `df` and the timestamp column is named `time`
df = df.withColumn('year', year(df.time))  # extract year
df = df.withColumn('month', month(df.time))  # extract month
df = df.withColumn('day', dayofmonth(df.time))  # extract day

In [ ]:
df.printSchema()

root
 |-- time: timestamp (nullable = true)
 |--  amount: double (nullable = true)
 |--  open: double (nullable = true)
 |--  high: double (nullable = true)
 |--  low: double (nullable = true)
 |--  close: double (nullable = true)
 |--  count: double (nullable = true)
 |--  vol: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)



In [ ]:
df.show()

+-------------------+------------------+-------+-------+-------+-------+------+------------------+----+-----+---+
|               time|            amount|   open|   high|    low|  close| count|               vol|year|month|day|
+-------------------+------------------+-------+-------+-------+-------+------+------------------+----+-----+---+
|2020-01-01 00:00:00|102.74380394536281| 7194.7| 7200.0|7186.19| 7200.0| 497.0| 738706.3461761171|2020|    1|  1|
|2020-01-01 00:01:00| 70.71166088549622| 7200.0|7219.58|7199.99| 7218.2| 448.0| 509693.2114912161|2020|    1|  1|
|2020-01-01 00:02:00|17.445959611852214|7218.21|7218.44|7213.09|7216.22| 248.0|   125889.77536392|2020|    1|  1|
|2020-01-01 00:03:00| 13.03460623212044|7216.21|7221.91|7215.48|7218.37| 300.0| 94091.10083095604|2020|    1|  1|
|2020-01-01 00:04:00|14.004799868097122|7218.37|7220.13|7210.01|7213.05| 279.0|   101042.53180219|2020|    1|  1|
|2020-01-01 00:05:00| 12.57289432271876|7213.48|7221.76|7212.62|7218.38| 189.0| 90727.99

In [ ]:
df.printSchema()

root
 |-- time: timestamp (nullable = true)
 |--  amount: double (nullable = true)
 |--  open: double (nullable = true)
 |--  high: double (nullable = true)
 |--  low: double (nullable = true)
 |--  close: double (nullable = true)
 |--  count: double (nullable = true)
 |--  vol: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)



In [ ]:
from pyspark.sql.functions import to_timestamp, year, month, dayofmonth
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor, RandomForestRegressor, GBTRegressor, LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.clustering import KMeans

In [ ]:
# Defining the feature column names
feature_cols = [' amount', ' open', ' high', ' low', ' count', ' vol', 'year', 'month', 'day']

In [ ]:
# Defining the target column name
target_col = ' close'

In [ ]:
# Creating a vector assembler to combine the feature columns into a single vector column
vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')

In [ ]:
# Transforming the DataFrame to include the features vector column
df = vector_assembler.transform(df)

In [ ]:
# Splitting the DataFrame into training and testing datasets
train_data, test_data = df.randomSplit([0.7, 0.3])

In [ ]:
# Training a Decision Tree Regressor model
dt = DecisionTreeRegressor(featuresCol='features', labelCol=target_col)
dt_model = dt.fit(train_data)
dt_predictions = dt_model.transform(test_data)

In [ ]:
# Training a Random Forest Regressor model
rf = RandomForestRegressor(featuresCol='features', labelCol=target_col)
rf_model = rf.fit(train_data)
rf_predictions = rf_model.transform(test_data)

In [ ]:
# Training a Gradient-Boosted Tree Regressor model
gbt = GBTRegressor(featuresCol='features', labelCol=target_col)
gbt_model = gbt.fit(train_data)
gbt_predictions = gbt_model.transform(test_data)

In [ ]:
# Training a Linear Regression model
lr = LinearRegression(featuresCol='features', labelCol=target_col)
lr_model = lr.fit(train_data)
lr_predictions = lr_model.transform(test_data)

In [ ]:
# Training a K-means clustering model
kmeans = KMeans(featuresCol='features', predictionCol='cluster', k=5)
kmeans_model = kmeans.fit(train_data)
train_data_clustered = kmeans_model.transform(train_data)
test_data_clustered = kmeans_model.transform(test_data)

In [ ]:
# Training a Decision Tree Regressor model on the clustered data
dt_clustered = DecisionTreeRegressor(featuresCol='features', labelCol=target_col)
dt_clustered_model = dt_clustered.fit(train_data_clustered)
dt_clustered_predictions = dt_clustered_model.transform(test_data_clustered)

In [ ]:
# Training a Random Forest Regressor model on the clustered data
rf_clustered = RandomForestRegressor(featuresCol='features', labelCol=target_col)
rf_clustered_model = rf_clustered.fit(train_data_clustered)
rf_clustered_predictions = rf_clustered_model.transform(test_data_clustered)

In [ ]:
# Training a Gradient-Boosted Tree Regressor model on the clustered data
gbt_clustered = GBTRegressor(featuresCol='features', labelCol=target_col)
gbt_clustered_model = gbt_clustered.fit(train_data_clustered)
gbt_clustered_predictions = gbt_clustered_model.transform(test_data_clustered)

In [ ]:
# Training a Linear Regression model on the clustered data
lr_clustered = LinearRegression(featuresCol='features', labelCol=target_col)
lr_clustered_model = lr_clustered.fit(train_data_clustered)
lr_clustered_predictions = lr_clustered_model.transform(test_data_clustered)

In [ ]:
# Evaluating the models' performance on the test data
evaluator = RegressionEvaluator(labelCol=target_col, predictionCol='prediction', metricName='rmse')

dt_rmse = evaluator.evaluate(dt_predictions)
rf_rmse = evaluator.evaluate(rf_predictions)
gbt_rmse = evaluator.evaluate(gbt_predictions)
lr_rmse = evaluator.evaluate(lr_predictions)

In [ ]:
#Training a KMeans clustering model
from pyspark.ml.clustering import KMeans

In [ ]:
# Training a KMeans clustering model
kmeans = KMeans(featuresCol='features', k=5)
kmeans_model = kmeans.fit(train_data)
kmeans_predictions = kmeans_model.transform(test_data)

In [ ]:
# Casting the prediction column to double
kmeans_predictions = kmeans_predictions.withColumn("prediction", kmeans_predictions["prediction"].cast("double"))

In [ ]:
# Evaluating the KMeans model's performance on the test data
evaluator = RegressionEvaluator(labelCol=target_col, predictionCol='prediction', metricName='rmse')
kmeans_rmse = evaluator.evaluate(kmeans_predictions)

In [ ]:
#Printing the models' evaluation metrics
print('Decision Tree Regressor Model: RMSE = %g' % dt_rmse)
print('Random Forest Regressor Model: RMSE = %g' % rf_rmse)
print('GBTRegressor Model: RMSE = %g' % gbt_rmse)
print('Linear Regression Model: RMSE = %g' % lr_rmse)
print('KMeans Clustering Model: RMSE = %g' % kmeans_rmse)

Decision Tree Regressor Model: RMSE = 628.423
Random Forest Regressor Model: RMSE = 773.024
GBTRegressor Model: RMSE = 444.304
Linear Regression Model: RMSE = 15.2454
KMeans Clustering Model: RMSE = 27720.4


In [ ]:
#..........................................................................................................................